In [1]:
from pathlib import Path
import json
import re
from tqdm import tqdm

settings = json.loads(Path('../../../settings.json').read_text())preprocessed_data_path = Path(settings['preprocessed_data_path'])data_path = Path('.').resolve()data_name = data_path.namesource_name = data_path.parent.namepreprocessed_dir = preprocessed_data_path/source_name/data_namesource_data_dir = preprocessed_dir/'preprocessed'
splits = ['train', 'valid']
tasks = ['대화 기반 페르소나 예측', '페르소나 채팅']
task_data_dir = preprocessed_dir/'preprocessed_task'
task_data_dir.mkdir(exist_ok=True)
for task in tasks:
    task_path = task_data_dir/task
    task_path.mkdir(exist_ok=True)
    
#### prepare for task preprocess



#### prepare for task preprocess end

#### task preprocess
for split in splits:
    source_data_dir_split = source_data_dir/split
    task_files = [(task_data_dir/task/f'{split}.jsonl').open('w', encoding='utf-8') for task in tasks]
    for source_data in tqdm(list(source_data_dir_split.iterdir()), desc=split):
        source_data = source_data.open()
        for line in source_data.readlines():
            line = json.loads(line)
            
            #### data preprocess
            personas = line['info']['personas']
            personas = [{'persona_id':persona['persona_id'], 
                         'persona':[p for p in persona['persona'] if p.pop('profile_id')]} for persona in personas]
            topic = line['info']['topic']
            utterances = line['utterances']
            utterances = [utterance for utterance in utterances if utterance.pop('terminate') == False]
            #### data preprocess end 
            
            
            #### 대화 기반 페르소나 예측
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            data['input']['topic'] = topic
            data['input']['dialog'] = utterances
            data['output']['personas'] = personas
            ## preprocess data from line end
            task_files[0].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 대화 기반 페르소나 예측 end
            
            
            #### 페르소나 채팅
            data = {'input': {}, 'output': {}}
            ## preprocess data from line
            data['input']['topic'] = topic
            data['input']['personas'] = personas
            data['output']['dialog'] = utterances
            
            ## preprocess data from line end
            task_files[1].write(json.dumps(data, ensure_ascii=False)+'\n')
            # print(json.dumps(data, indent=4, ensure_ascii=False))
            #### 페르소나 채팅 end
            
            
            
    #         break
    #     break
    # break

    for path in task_files:
        path.close()      
        
#### task preprocess end      

valid: 100%|██████████| 20/20 [00:00<00:00, 50.30it/s]
